In [1]:
import nltk
nltk.download('stopwords')

import os
import pickle

from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  mean_squared_error, mean_absolute_error

import numpy as np

[nltk_data] Downloading package stopwords to /home/alloky-
[nltk_data]     wsl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
texts = []
likes = []
for filename in tqdm(os.listdir('clean_files')):
    one_file = pickle.load(open(f'clean_files/{filename}', 'rb'))
    text = one_file['text']
    like = (-1 * int(one_file['likes'][1:])) if one_file['likes'][0] == '–' else int(one_file['likes'])
    texts.append(text)
    likes.append(like)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(texts, likes, test_size=0.33, random_state=42)

In [4]:
vectorizer = TfidfVectorizer(
    max_features=2048, lowercase=True, analyzer='word',
    stop_words=stopwords.words('russian'), ngram_range=(1,3),dtype=np.float32
)
train_vects = vectorizer.fit_transform(X_train)
test_vects = vectorizer.transform(X_test)

In [5]:
regressor = LinearRegression().fit(train_vects, y_train)

In [6]:
def evaluate(predictor, X_train, X_test, y_train, y_test):
    predicted_train = predictor.predict(X_train)
    predicted_test = predictor.predict(X_test)
    
    
    print('MSE train', mean_squared_error(predicted_train, y_train))
    print('MAE train', mean_absolute_error(predicted_train, y_train))
    print()
    print('MSE test', mean_squared_error(predicted_test, y_test))
    print('MAE train', mean_absolute_error(predicted_test, y_test))
    
#     print()
#     for i in range(10):
#         print(f"Example {i} : real = {y_test[i]} , predicted = {predicted_test[i]}")
#         print(X_test[i])

In [7]:
evaluate(regressor, train_vects, test_vects, y_train, y_test)

MSE train 235.46784600835025
MAE train 10.819642730285754

MSE test 629.9881548914256
MAE train 17.332090533725896
